In [693]:
import string
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
import os
import pandas as pd
import numpy as np
import numpy as np

def otsu_thresholding(importancia: np.ndarray) -> float:
    hist, _ = np.histogram(importancia, bins=400, range=(importancia[-1], importancia[0]))  # Calcular el histograma
    rango = (importancia[0] - importancia[-1])/400
    hist = hist / np.sum(hist)  # Normalizar el histograma para obtener probabilidades
    max_sigma = 0
    threshold = 0

    for t in range(1, len(hist)):
        w0 = np.sum(hist[:t])  # Probabilidad acumulada para la clase 0
        w1 = 1 - w0  # Probabilidad acumulada para la clase 1
        if w0 == 0 or w1 == 0:
            continue

        mu0 = np.sum(np.arange(t) * hist[:t]) / w0  # Media ponderada para la clase 0
        mu1 = np.sum(np.arange(t, len(hist)) * hist[t:]) / w1  # Media ponderada para la clase 1

        sigma = w0 * w1 * ((mu0 - mu1) ** 2)  # Calcular varianza entre clases

        if sigma > max_sigma:
            max_sigma = sigma
            threshold = t
    return threshold * rango


def leerArchivos(carpeta:str) -> list[str]:
    cuerpo = []
    archivos_txt = [archivo for archivo in os.listdir(carpeta) if archivo.endswith('.txt')]
    for archivo in archivos_txt:
        ruta_completa = os.path.join(carpeta, archivo)
        with open(ruta_completa, 'r',encoding="utf-8") as file:
            contenido = file.read()
            cuerpo.append(contenido)
    return cuerpo

def normalizarTexto(texto:str) -> (str,list[str],list[str]):
    stopwords_es = set(stopwords.words('spanish'))
    signos_puntuacion = string.punctuation + '¡¿'
    # Eliminar signos de puntuación
    oraciones = texto.split(".")[:-1]
    oraciones_normalizadas = []
    oraciones_stop = []
    texto_normalizado = ""
    for oracion in oraciones:
        texto_sin_puntuacion = ''.join([caracter for caracter in oracion if caracter not in signos_puntuacion])
        # Tokenizar el texto en palabras
        palabras = word_tokenize(texto_sin_puntuacion, language='spanish')
        palabras = [palabra.lower() for palabra in palabras]
        # Eliminar stopwords
        palabras_sin_stopwords = [palabra for palabra in palabras if palabra not in stopwords_es]
        oraciones_stop.append(' '.join(palabras))
        oraciones_normalizadas.append(' '.join(palabras_sin_stopwords))
        texto_normalizado = texto_normalizado+ ' '.join(palabras_sin_stopwords)
    return texto_normalizado, oraciones_normalizadas,oraciones_stop

def normalizarCuerpo(cuerpo:list[str])->list[str]:
    cuerpo_normalizado = []
    oraciones_normalizadas = []
    oraciones_stop = []
    for documento in cuerpo:
        c_norm,oraciones_cuerpo,oraciones_cuerpo_stop = normalizarTexto(documento)
        cuerpo_normalizado.append(c_norm)
        oraciones_normalizadas.append(oraciones_cuerpo)
        oraciones_stop.append(oraciones_cuerpo_stop)
    return cuerpo_normalizado,oraciones_normalizadas,oraciones_stop

def calcularITFDTCuerpo(cuerpo:list[str])->pd.DataFrame:
    vectorizador_tfidf = TfidfVectorizer()
    matriz_tfidf = vectorizador_tfidf.fit_transform(cuerpo)
    palabras = vectorizador_tfidf.get_feature_names_out()
    dataframes = []
    for doc_idx, documento in enumerate(cuerpo):
        data = []
        palabras_doc = documento.split()
        palabras_doc = [palabra for palabra in palabras if palabra in palabras_doc]
        for palabra in palabras_doc:
            index_palabra = vectorizador_tfidf.vocabulary_.get(palabra)
            valor_idf = vectorizador_tfidf.idf_[index_palabra]
            valor_tfidf = matriz_tfidf[:, index_palabra].toarray().flatten()[doc_idx]
            valor_tf = valor_tfidf/valor_idf
            data.append([
                palabra,
                doc_idx + 1,
                valor_tf,
                valor_idf,
                valor_tfidf
            ])
        dataframes.append(pd.DataFrame(data, columns=["palabra", "documento", "TF", "IDF", "TFIDF"]))
    return dataframes


def obtenerMejoresPuestos(dfdocumento:pd.DataFrame,columna:str,cantidad:int=10,tipo:str="cantidad")->pd.DataFrame:
    if(tipo == "cantidad"):
        return dfdocumento.sort_values(by=columna, ascending=False).head(cantidad)
    else:
        cuerpo_ordenado = dfdocumento.sort_values(by=columna, ascending=False)
        t = otsu_thresholding(cuerpo_ordenado[columna].to_numpy())
        return cuerpo_ordenado[cuerpo_ordenado[columna] >= t]

def similitudOracionesEmbeddings(cuerpo_oracion):
    cuerpo_oracion_ranked = []
    for oraciones in cuerpo_oracion:
        word_embeddings_model = Word2Vec(oraciones, vector_size=100, window=5, min_count=1, sg=0)
        # Calcular los embeddings de las oraciones
        sentence_embeddings = []
        for oracion in oraciones:
            palabras = oracion.split()
            embedding_oracion = np.mean([word_embeddings_model.wv[palabra] for palabra in palabras if palabra in word_embeddings_model.wv] or [np.zeros(100)], axis=0)
            sentence_embeddings.append(embedding_oracion)
        # Calcula la similitud entre oraciones
        similarity_matrix = np.zeros([len(oraciones), len(oraciones)])
        for i in range(len(oraciones)):
            for j in range(len(oraciones)):
                if(i==j):
                    similarity_matrix[i][j]=0
                elif((np.linalg.norm(sentence_embeddings[i]) * np.linalg.norm(sentence_embeddings[j])) == 0):
                    similarity_matrix[i][j] = 0
                else:
                    similarity_matrix[i][j] = np.dot(sentence_embeddings[i], sentence_embeddings[j]) / (np.linalg.norm(sentence_embeddings[i]) * np.linalg.norm(sentence_embeddings[j]))
        ranking_oraciones = np.argsort(np.sum(similarity_matrix, axis=1))[::-1]
        cuerpo_oracion_ranked.append([oraciones[idx] for idx in ranking_oraciones])
    return cuerpo_oracion_ranked

def generarNGramasDocumento(documento:list[str],n):
    ngramas = []
    fngramas = []
    for oracion in documento:
        tokens = word_tokenize(oracion)
        ngramas.append(list(ngrams(tokens,n)))
    for oracion in ngramas:
        longitud = len(oracion)
        fngrama = {}
        for grama in oracion:
            if grama in fngrama:
                fngrama[grama] += 1/longitud
            else:
                fngrama[grama] = 1/longitud
        fngramas.append(fngrama)
    return fngramas
    
def generarNGramasCuerpo(oraciones_cuerpo:list[list[str]],n:int=2):
    fnagramasCuerpo = []
    for documento in oraciones_cuerpo:
        fngramas = generarNGramasDocumento(documento,n)
        fnagramasCuerpo.append(fngramas)
    return fnagramasCuerpo

def encontrarMejorConector():pass

In [694]:
cuerpo = leerArchivos("documentos")
cuerpo,oraciones_cuerpo,oraciones_stop = normalizarCuerpo(cuerpo)
df_cuerpo = calcularITFDTCuerpo(cuerpo)
df_cuerpo[0].head()

,palabra,documento,TF,IDF,TFIDF
0,1980,1,0.025994,1.0,0.025994
1,acceso,1,0.025994,1.0,0.025994
2,acción,1,0.025994,1.0,0.025994
3,adaptar,1,0.025994,1.0,0.025994
4,adaptarse,1,0.025994,1.0,0.025994


In [695]:
oraciones_cuerpo_ranked = similitudOracionesEmbeddings(oraciones_cuerpo)

In [696]:
fngramasCuerpo = generarNGramasCuerpo(oraciones_stop)
print(fngramasCuerpo[0])

[{('el', 'aprendizaje'): 0.030303030303030304, ('aprendizaje', 'automático'): 0.030303030303030304, ('automático', 'un'): 0.030303030303030304, ('un', 'subcampo'): 0.030303030303030304, ('subcampo', 'de'): 0.030303030303030304, ('de', 'la'): 0.06060606060606061, ('la', 'inteligencia'): 0.030303030303030304, ('inteligencia', 'artificial'): 0.030303030303030304, ('artificial', 'es'): 0.030303030303030304, ('es', 'un'): 0.030303030303030304, ('un', 'enfoque'): 0.030303030303030304, ('enfoque', 'revolucionario'): 0.030303030303030304, ('revolucionario', 'que'): 0.030303030303030304, ('que', 'permite'): 0.030303030303030304, ('permite', 'a'): 0.030303030303030304, ('a', 'las'): 0.030303030303030304, ('las', 'máquinas'): 0.030303030303030304, ('máquinas', 'aprender'): 0.030303030303030304, ('aprender', 'y'): 0.030303030303030304, ('y', 'mejorar'): 0.030303030303030304, ('mejorar', 'a'): 0.030303030303030304, ('a', 'partir'): 0.030303030303030304, ('partir', 'de'): 0.030303030303030304, ('la'

In [697]:
def generarResumenDocumento(df_documento,oraciones_cuerpo_ranked,fngramas):
    cantidad_oraciones = len(oraciones_cuerpo_ranked)
    mejores_puestos = obtenerMejoresPuestos(df_documento,"TFIDF",tipo="otsu")
    palabras = mejores_puestos["palabra"].to_numpy()
    oraciones_tfidf = np.zeros((cantidad_oraciones))
    for idx,oracion in enumerate(oraciones_cuerpo_ranked):
        palabras_oracion = oracion.split()
        for palabra in palabras_oracion:
            if palabra in palabras:
                oraciones_tfidf[idx] +=1
    oraciones_redundates = oraciones_tfidf[:cantidad_oraciones*2//3] 
    oraciones_extra = oraciones_tfidf[cantidad_oraciones*2//3:]
    oraciones_redundates_sort =np.argsort(oraciones_redundates)[::-1]
    oraciones_extra_sort = np.argsort(oraciones_extra)[::-1]
    resumen = []
    for i in range(5):
        resumen.append(oraciones_cuerpo_ranked[oraciones_redundates_sort[i]])
    for i in range(1):
        resumen.append(oraciones_cuerpo_ranked[oraciones_extra_sort[i]])
    return ', '.join(resumen)

generarResumenDocumento(df_cuerpo[0],oraciones_cuerpo_ranked[0],fngramasCuerpo[0])

'aprendizaje supervisado algoritmo aprende partir conjunto datos etiquetado decir conjunto datos “ respuesta correcta ” conoce antemano, aprendizaje supervisado lado algoritmo aprende partir conjunto datos etiquetado, aprendizaje profundo tipo aprendizaje automático utiliza redes neuronales artificiales muchas capas ahí término “ profundo ” aprender partir datos, aprendizaje automático puede ser semisupervisado combinación enfoques supervisado supervisado, aprendizaje automático puede ser supervisado supervisado, sistemas utilizan algoritmos aprendizaje automático aprender interacciones usuarios adaptar contenido consecuencia'